In [2]:
import sys
sys.path.append('..')
from osp import *

In [ ]:
STASH_WORD_FREQS = HashStash('osp_word_freqs', append_mode=False)
PATH_WORDFREQS_TSV = os.path.join(PATH_DATA, 'raw', 'word_freqs.tsv')
PATH_WORDFREQS_TSV_NL = 62141323

def gen_word_freqs_raw(lim=None):
    ndone=0
    with open(PATH_WORDFREQS_TSV, 'a+') as f:
        for text_id, text_counts in tqdm(STASH_COUNTS.items(), total=len(STASH_COUNTS),desc='gen_word_freqs',position=0):
            total_count = sum(text_counts.values())
            for word, count in text_counts.items():
                f.write(f'{word}\t{text_id}\t{count}\n')

In [6]:
gen_word_freqs(lim=None)

gen_word_freqs: 100%|██████████| 58126/58126 [00:35<00:00, 1625.93it/s]


In [20]:
# !tail ../data/raw/word_freqs.tsv

In [22]:
def iter_word_freqs_raw():
    with open(PATH_WORDFREQS_TSV, 'r') as f:
        for line in tqdm(f, total=PATH_WORDFREQS_TSV_NL, desc='get_word_freqs', position=0):
            ln = line.strip()
            if not ln or not ln.count('\t')==2: continue
            word, text_id, count = ln.split('\t')
            yield word, text_id, int(count)

In [42]:
# next(iter_word_freqs_raw())

In [70]:
def load_word_freqs(words):
    ld = []
    for w in words:
        if w in STASH_WORD_FREQS:
            for text_id, count in STASH_WORD_FREQS[w].items():
                ld.append({'word': w, 'text_id': text_id, 'count': count})
    return pd.DataFrame(ld)



In [71]:
load_word_freqs(['of'])

,word,text_id,count
0,of,phil/10.2307/2375809,205
1,of,phil/10.2307/45238188,309
2,of,phil/10.2307/20115538,140
3,of,phil/10.2307/40380502,223
4,of,phil/10.2307/24659549,246
...,...,...,...
56547,of,lit/2872592,317
56548,of,phil/10.2307/42964393,2
56549,of,phil/10.2307/27293693,166
56550,of,lit/3718567,12


In [76]:
# STASH_WORD_FREQS.clear()

def get_word_freqs(words, force=False):
    words_done_df = load_word_freqs(words)
    words_not_done = [w for w in words if w not in words_done_df.word.unique()] if len(words_done_df) else words
    if not words_not_done:
        return words_done_df
    
    ld = words_done_df.to_dict(orient='records')
    word2textid2count = defaultdict(lambda: defaultdict(int))

    for word, text_id, count in iter_word_freqs_raw():
        if word in words_not_done:
            word2textid2count[word][text_id] += count
    
    # textid2total = Counter()
    for word,text_id_count in word2textid2count.items():
        STASH_WORD_FREQS[word] = text_id_count
        for text_id, count in text_id_count.items():
            # textid2total[text_id] += count
            ld.append(
                {
                    'word': word,
                    'text_id': text_id,
                    'count': count,
                }
            )
    odf = pd.DataFrame(ld)
    # odf['total'] = odf['text_id'].map(textid2total)
    return odf


In [80]:
get_word_freqs(['of','the'])

,word,text_id,count
0,of,phil/10.2307/2375809,205
1,of,phil/10.2307/45238188,309
2,of,phil/10.2307/20115538,140
3,of,phil/10.2307/40380502,223
4,of,phil/10.2307/24659549,246
...,...,...,...
112924,the,lit/2872592,461
112925,the,phil/10.2307/42964393,7
112926,the,phil/10.2307/27293693,256
112927,the,lit/3718567,19


In [81]:
get_word_freqs(get_non_content_words())

get_word_freqs: 100%|██████████| 62141323/62141323 [05:13<00:00, 198133.18it/s]


,word,text_id,count
0,the,phil/10.2307/2375809,311
1,the,phil/10.2307/45238188,375
2,the,phil/10.2307/20115538,189
3,the,phil/10.2307/40380502,278
4,the,phil/10.2307/24659549,310
...,...,...,...
7327972,yippee,lit/20057577,1
7327973,northeastward,lit/1343465,1
7327974,southeastward,lit/2871862,1
7327975,some-where,lit/3732327,1


In [54]:
# def get_word_freqs(words):
#     ld = []
#     textid2total = Counter()
#     with open(PATH_WORDFREQS_TSV, 'r') as f:
#         for line in tqdm(f, total=PATH_WORDFREQS_TSV_NL, desc='get_word_freqs', position=0):
#             ln = line.strip()
#             if not ln or not ln.count('\t')==2: continue
#             word, text_id, count = ln.split('\t')
#             textid2total[text_id] += int(count)
#             if word in words:
#                 ld.append(
#                     {
#                         'word': word,
#                         'text_id': text_id,
#                         'count': count,
#                     }
#                 )
#     odf = pd.DataFrame(ld)
#     odf['total'] = odf['text_id'].map(textid2total)
#     return odf


In [53]:
get_word_freqs(['the'])

get_word_freqs:  31%|███       | 19403286/62141323 [00:08<00:18, 2365259.54it/s]


KeyboardInterrupt: 

In [12]:
!wc -l ../data/raw/word_freqs.tsv

 62141323 ../data/raw/word_freqs.tsv
